In [1]:
import pandas as pd
import dask as dask
from dask import dataframe as dd
import numpy as np
from dask_jobqueue import SLURMCluster
from dask.distributed import Client
import dask.array as da
import functools
import sys
import ast
from struct import *
import pickle
import glob
import random
import seaborn as snsa
import ipywidgets as widgets
import matplotlib
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import scipy.stats as st

In [7]:
file = open('instance_files/instance_db_aggagg_flat_returnclus1', 'rb')
bflat = pickle.load(file)
#file = open('instance_files/instance_db_aggagg_rand_returnclus1', 'rb')
#brand = pickle.load(file)

In [8]:
#FLAT
#a=z b=y c=x

#dunnavan: a>= b >= c
#in our case z >= y >= x

nclusters = 300
rxs_flat = np.zeros((20,20,nclusters))
rys_flat = np.zeros((20,20,nclusters))
rzs_flat = np.zeros((20,20,nclusters))
phi2Ds_flat = np.zeros((20,20,nclusters))
cplxs_flat = np.zeros((20,20,nclusters))
dds_flat = np.zeros((20,20,nclusters))

counter=0       
for phi in range(20):
    for r in range(20):
        rxs_flat[phi, r, :] = np.array(bflat[counter][0])[:,0]
        rys_flat[phi, r, :] = np.array(bflat[counter][0])[:,1]
        rzs_flat[phi, r, :] = np.array(bflat[counter][0])[:,2]
        phi2Ds_flat[phi, r, :] = bflat[counter][1]
        cplxs_flat[phi, r, :] = bflat[counter][2]
        dds_flat[phi, r, :] = bflat[counter][3]
        counter+=1
phiba_flat = rys_flat/rzs_flat
phica_flat = rxs_flat/rzs_flat
np.shape(phica_flat)

(20, 20, 300)

In [ ]:
#RAND
file = open('instance_files/instance_db_aggagg_rand_returnclus', 'rb')
b2 = pickle.load(file)

nclusters = 300
rxs_rand = np.zeros((20,20,nclusters))
rys_rand = np.zeros((20,20,nclusters))
rzs_rand = np.zeros((20,20,nclusters))
phi2Ds_rand = np.zeros((20,20,nclusters))
cplxs_rand = np.zeros((20,20,nclusters))
dds_rand = np.zeros((20,20,nclusters))

counter=0
for phi in range(20):
    for r in range(20):
        rxs_rand[phi, r, :] = np.array(brand[counter][0])[:,0]
        rys_rand[phi, r, :] = np.array(brand[counter][0])[:,1]
        rzs_rand[phi, r, :] = np.array(brand[counter][0])[:,2]
        phi2Ds_rand[phi, r, :] = brand[counter][1]
        cplxs_rand[phi, r, :] = brand[counter][2]
        dds_rand[phi, r, :] = brand[counter][3]
        counter+=1
phiba_rand = rys_rand/rzs_rand
phica_rand = rxs_rand/rzs_rand
np.shape(phica_rand)

In [48]:
class IceClusterBatch():
    """A collection of IceCluster objects."""
    
    def __init__(self, nclusters, rxs, rys, rzs, phi2Ds, cplxs, dds, numaspectratios, numrs):

        self.nclusters=nclusters
        self.rxs = rxs
        self.rys = rys
        self.rzs = rzs
        self.phi2Ds = phi2Ds
        self.cplxs = cplxs
        self.dds = dds
        self.numaspectratios = numaspectratios
        self.numrs = numrs


    def calculate_error(self, data, ch):
        mean = np.mean(data)
        std = np.std(data)
        shape = (mean/std)**2
        scale = (std**2)/mean
        shapech = mean/(self.numaspectratios*ch)

        pos_error = mean + std
        neg_error = mean - std

        min_data = min(data)
        max_data = max(data)

        return(pos_error, neg_error, min_data, max_data, mean)        


    def fit_distribution(self, data, normed = True, facecolor='navy', alpha=1.0,**kwargs):

        """Model data by finding best fit distribution to data"""
        # Get histogram of original data

        #fig = plt.figure(figsize=(5,7))
        #ax = plt.subplot(131)

        data = np.array(data)
        data[np.isinf(data)] = min(data)
        data[np.isnan(data)] = min(data)
        if np.isinf(data).any():
            print('inf True')
        y, x = np.histogram(data, density=True)
        #print('x hist',x)
        xx = (x + np.roll(x, -1))[:-1] / 2.0

        
        params = st.gamma.fit(data)
        gamma_params = params
        arg = gamma_params[:-2]
        #fig = plt.figure(figsize=(10,7))
        #ax = fig.add_subplot(111)
        #n, bins, patches = plt.hist(data, bins=bins, normed=True,
        #                            color='navy',range=(min(data), max(data)),**kwargs)
        #pdf = self.make_pdf(distribution, gamma_params)
        pdf = st.gamma.pdf(xx, loc=gamma_params[-2], scale=gamma_params[-1], *arg)
        indmax = np.argmax(pdf)  #FIRST index where the highest prob occurs
        gammach_var = x[indmax] #characteristic of the distribution
        #ax = plt.plot(xx, pdf, lw=5, color='darkorange')
        #plt.show()

        return gammach_var

In [ ]:
#find characteristic of gamma distribution
numaspectratios=np.shape(rxs_flat)[0]
numrs=np.shape(rxs_flat)[1]
batch = IceClusterBatch(nclusters, rxs_flat, rys_flat, rzs_flat, phi2Ds_flat, \
                        cplxs_flat, dds_flat, numaspectratios, numrs)

rxs_flat_ch = np.empty((numaspectratios, numrs), dtype=np.float64)
rzs_flat_ch = np.empty((numaspectratios, numrs), dtype=np.float64)
rzs_flat_mean = np.empty((numaspectratios, numrs), dtype=np.float64)
dds_flat_ch = np.empty((numaspectratios, numrs), dtype=np.float64)

for i in range(numaspectratios):
    for r in range(numrs):
        for c, data in enumerate([rxs_flat, rzs_flat, dds_flat]):
            if c == 0:
                rxs_flat_ch[i,r] = batch.fit_distribution(data[i,r,:])
            if c == 1:
                rzs_flat_ch[i,r] = batch.fit_distribution(data[i,r,:])
                _,_,_,_,rzs_flat_mean[i,r] = batch.calculate_error(data[i,r,:], rzs_flat_ch[i,r])
            if c == 2:
                dds_flat_ch[i,r] = batch.fit_distribution(data[i,r,:])
                

In [ ]:
#write to file:

%store -r phi_bins
%store -r r_bins

with open("lookup_tables/minorax_flat_ch.dat","w+") as file1:
    with open("lookup_tables/majorax_flat_ch.dat","w+") as file2:
        with open("lookup_tables/majorax_flat_mean.dat","w+") as file3:
            with open("lookup_tables/dd_flat_ch.dat","w+") as file4:

                    for phi in range(len(phi_bins)-1):
                        for r in range(len(r_bins)-1):
                            file1.write('%.2f, %.2f,%.2f,%.2f,%.2f \n' %(phi_bins[phi], phi_bins[phi+1], r_bins[r], r_bins[r+1], rxs_flat_ch[phi,r]))
                            file2.write('%.2f, %.2f,%.2f,%.2f,%.2f \n' %(phi_bins[phi], phi_bins[phi+1], r_bins[r], r_bins[r+1], rzs_flat_ch[phi,r]))
                            file3.write('%.2f, %.2f,%.2f,%.2f,%.2f \n' %(phi_bins[phi], phi_bins[phi+1], r_bins[r], r_bins[r+1], rzs_flat_mean[phi,r]))
                            file4.write('%.2f, %.2f,%.2f,%.2f,%.2f \n' %(phi_bins[phi], phi_bins[phi+1], r_bins[r], r_bins[r+1], dds_flat_ch[phi,r]))

file1.close()
file2.close() 
file3.close()
file4.close()